In [1]:
import os

import pymongo

from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.support.ui import Select

import pandas as pd

import re
from pprint import pprint
from datetime import datetime
import time

import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey, Date, DateTime, Boolean, BigInteger
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy import inspect

In [3]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.social_blade

In [4]:
collection = mongoDb.social_blade_test

In [5]:
executable_path = {'executable_path': '/Users/soria/Anaconda3/Scripts/chromedriver'}
browser = Browser('chrome', **executable_path)


In [6]:
if mongoDb.social_blade_test.find_one():
    html = mongoDb.social_blade_test.find_one()['social_blade_html']
else:
    url = "https://socialblade.com/youtube/user/GentleWhispering/monthly"
    browser.visit(url)
    html = browser.html
test_html_dict = {'social_blade_url': url, 'social_blade_html':html}
collection.update_one({}, {'$set': test_html_dict}, upsert=True)

blade_soup = bs(html, 'html.parser')


In [7]:
sortdiv = blade_soup.find('div', {'id': 'sort-by'})
sortdiv.next_sibling.next_sibling.next_sibling.next_sibling


AttributeError: 'NoneType' object has no attribute 'next_sibling'

In [ ]:
divs = blade_soup.find_all('div')
for index, div in enumerate(divs):
    print(f"{index}")
    print("========================")
    print(f"{div}")

In [ ]:
the_div = divs[177:178]
the_div

In [ ]:
spans = the_div[0].find_all('span')
for span in spans:
    print(span.text)

In [ ]:
anchors = the_div[0].find_all('a')
for anchor in anchors:
    print(anchor['href'])

In [3]:
url = "https://socialblade.com/youtube/user/GentleWhispering/monthly"
browser.visit(url)
html = browser.html
blade_soup = bs(html, 'html.parser')


In [8]:
script_divs = blade_soup.find_all('script', {'type': 'text/javascript'})
# for index, script in enumerate(script_divs[9:15]):
#     print(index)
#     print('============================')
#     print(script)
script_divs = script_divs[9:15]


In [13]:
len(raw_text)

23905

In [14]:
len(front_strip)

23905

In [28]:
raw_text = script_divs[0].text
front_strip = re.sub(r'\Subs\\\\n"(.*)', "", raw_text)
front_strip

'\t\t\n\tg = new Dygraph(\n\n\t// containing div\n\tdocument.getElementById(\'DailySubscribers\'),\n\t// CSV or path to a CSV file.\n\t"Date,Daily Subs\\n" + "2016-02-15,702\\n" + "2016-02-16,455\\n" + "2016-02-17,549\\n" + "2016-02-18,576\\n" + "2016-02-19,563\\n" + "2016-02-20,633\\n" + "2016-02-21,533\\n" + "2016-02-22,400\\n" + "2016-02-23,619\\n" + "2016-02-24,654\\n" + "2016-02-25,592\\n" + "2016-02-26,587\\n" + "2016-02-27,592\\n" + "2016-02-28,666\\n" + "2016-02-29,484\\n" + "2016-03-01,592\\n" + "2016-03-02,583\\n" + "2016-03-03,488\\n" + "2016-03-04,472\\n" + "2016-03-05,528\\n" + "2016-03-06,609\\n" + "2016-03-07,572\\n" + "2016-03-08,701\\n" + "2016-03-09,665\\n" + "2016-03-10,662\\n" + "2016-03-11,695\\n" + "2016-03-12,563\\n" + "2016-03-13,650\\n" + "2016-03-14,646\\n" + "2016-03-15,633\\n" + "2016-03-16,539\\n" + "2016-03-17,650\\n" + "2016-03-18,600\\n" + "2016-03-19,709\\n" + "2016-03-20,503\\n" + "2016-03-21,766\\n" + "2016-03-22,880\\n" + "2016-03-23,594\\n" + "2016-

In [ ]:

raw_text[139:-330]

In [ ]:
raw_text[139:-497]

In [ ]:
test_list = raw_text[139:-397].split('+')
stripped = [test.strip('"\\n') for test in test_list]
strip_test = re.sub(f'\\\\n"', "", test_list[2])
# strip_test = re.sub(f'"', "", strip_test)
# test_date_val = strip_test.split(',')
# test_date_val[1]
test_list

In [49]:
def dataframer(blob, val_type):
    string_list = blob.split('+')
    string_list = string_list[1:]
    stripped = [string.strip('"\\n') for string in string_list]
    stripped = [(re.sub(f'\\\\n"', "", string)) for string in stripped]
    stripped = [(re.sub(f'"', "", string)) for string in stripped]
    dict_list = []
    for strip in stripped:
        date_val = strip.split(',')
        print(date_val)
        date = date_val[0]
        print(date)
        val = date_val[1]
        print(val)
        dict_list.append({'date': date, val_type: val})
    return dict_list

In [57]:
script_divs[6].text.split('+')

IndexError: list index out of range

In [58]:
dataframer(script_divs[0].text, 'daily_subs')


[' 2016-02-15', '702 ']
 2016-02-15
702 
[' 2016-02-16', '455 ']
 2016-02-16
455 
[' 2016-02-17', '549 ']
 2016-02-17
549 
[' 2016-02-18', '576 ']
 2016-02-18
576 
[' 2016-02-19', '563 ']
 2016-02-19
563 
[' 2016-02-20', '633 ']
 2016-02-20
633 
[' 2016-02-21', '533 ']
 2016-02-21
533 
[' 2016-02-22', '400 ']
 2016-02-22
400 
[' 2016-02-23', '619 ']
 2016-02-23
619 
[' 2016-02-24', '654 ']
 2016-02-24
654 
[' 2016-02-25', '592 ']
 2016-02-25
592 
[' 2016-02-26', '587 ']
 2016-02-26
587 
[' 2016-02-27', '592 ']
 2016-02-27
592 
[' 2016-02-28', '666 ']
 2016-02-28
666 
[' 2016-02-29', '484 ']
 2016-02-29
484 
[' 2016-03-01', '592 ']
 2016-03-01
592 
[' 2016-03-02', '583 ']
 2016-03-02
583 
[' 2016-03-03', '488 ']
 2016-03-03
488 
[' 2016-03-04', '472 ']
 2016-03-04
472 
[' 2016-03-05', '528 ']
 2016-03-05
528 
[' 2016-03-06', '609 ']
 2016-03-06
609 
[' 2016-03-07', '572 ']
 2016-03-07
572 
[' 2016-03-08', '701 ']
 2016-03-08
701 
[' 2016-03-09', '665 ']
 2016-03-09
665 
[' 2016-03-10', 

[' 2017-08-15', '756 ']
 2017-08-15
756 
[' 2017-08-16', '722 ']
 2017-08-16
722 
[' 2017-08-17', '790 ']
 2017-08-17
790 
[' 2017-08-18', '650 ']
 2017-08-18
650 
[' 2017-08-19', '579 ']
 2017-08-19
579 
[' 2017-08-20', '597 ']
 2017-08-20
597 
[' 2017-08-21', '366 ']
 2017-08-21
366 
[' 2017-08-22', '678 ']
 2017-08-22
678 
[' 2017-08-23', '749 ']
 2017-08-23
749 
[' 2017-08-24', '485 ']
 2017-08-24
485 
[' 2017-08-25', '560 ']
 2017-08-25
560 
[' 2017-08-26', '635 ']
 2017-08-26
635 
[' 2017-08-27', '457 ']
 2017-08-27
457 
[' 2017-08-28', '739 ']
 2017-08-28
739 
[' 2017-08-29', '910 ']
 2017-08-29
910 
[' 2017-08-30', '786 ']
 2017-08-30
786 
[' 2017-08-31', '722 ']
 2017-08-31
722 
[' 2017-09-01', '590 ']
 2017-09-01
590 
[' 2017-09-02', '625 ']
 2017-09-02
625 
[' 2017-09-03', '618 ']
 2017-09-03
618 
[' 2017-09-04', '684 ']
 2017-09-04
684 
[' 2017-09-05', '667 ']
 2017-09-05
667 
[' 2017-09-06', '803 ']
 2017-09-06
803 
[' 2017-09-07', '558 ']
 2017-09-07
558 
[' 2017-09-08', 

 2018-08-24
1644 
[' 2018-08-25', '1932 ']
 2018-08-25
1932 
[' 2018-08-26', '1691 ']
 2018-08-26
1691 
[' 2018-08-27', '1540 ']
 2018-08-27
1540 
[' 2018-08-28', '1690 ']
 2018-08-28
1690 
[' 2018-08-29', '1306 ']
 2018-08-29
1306 
[' 2018-08-30', '1232 ']
 2018-08-30
1232 
[' 2018-08-31', '1287 ']
 2018-08-31
1287 
[' 2018-09-01', '1339 ']
 2018-09-01
1339 
[' 2018-09-02', '1210 ']
 2018-09-02
1210 
[' 2018-09-03', '1742 ']
 2018-09-03
1742 
[' 2018-09-04', '2286 ']
 2018-09-04
2286 
[' 2018-09-05', '2106 ']
 2018-09-05
2106 
[' 2018-09-06', '1676 ']
 2018-09-06
1676 
[' 2018-09-07', '1303 ']
 2018-09-07
1303 
[' 2018-09-08', '1426 ']
 2018-09-08
1426 
[' 2018-09-09', '1433 ']
 2018-09-09
1433 
[' 2018-09-10', '1479 ']
 2018-09-10
1479 
[' 2018-09-11', '1780 ']
 2018-09-11
1780 
[' 2018-09-12', '1316 ']
 2018-09-12
1316 
[' 2018-09-13', '1133 ']
 2018-09-13
1133 
[' 2018-09-14', '1242 ']
 2018-09-14
1242 
[' 2018-09-15', '1379 ']
 2018-09-15
1379 
[' 2018-09-16', '1208 ']
 2018-09-16

[{'date': ' 2016-02-15', 'daily_subs': '702 '},
 {'date': ' 2016-02-16', 'daily_subs': '455 '},
 {'date': ' 2016-02-17', 'daily_subs': '549 '},
 {'date': ' 2016-02-18', 'daily_subs': '576 '},
 {'date': ' 2016-02-19', 'daily_subs': '563 '},
 {'date': ' 2016-02-20', 'daily_subs': '633 '},
 {'date': ' 2016-02-21', 'daily_subs': '533 '},
 {'date': ' 2016-02-22', 'daily_subs': '400 '},
 {'date': ' 2016-02-23', 'daily_subs': '619 '},
 {'date': ' 2016-02-24', 'daily_subs': '654 '},
 {'date': ' 2016-02-25', 'daily_subs': '592 '},
 {'date': ' 2016-02-26', 'daily_subs': '587 '},
 {'date': ' 2016-02-27', 'daily_subs': '592 '},
 {'date': ' 2016-02-28', 'daily_subs': '666 '},
 {'date': ' 2016-02-29', 'daily_subs': '484 '},
 {'date': ' 2016-03-01', 'daily_subs': '592 '},
 {'date': ' 2016-03-02', 'daily_subs': '583 '},
 {'date': ' 2016-03-03', 'daily_subs': '488 '},
 {'date': ' 2016-03-04', 'daily_subs': '472 '},
 {'date': ' 2016-03-05', 'daily_subs': '528 '},
 {'date': ' 2016-03-06', 'daily_subs': '

In [22]:
back_strip = re.search(r'^(.*?)Sub', raw_text)
back_strip

In [47]:
test = re.search(r'.+?(?=Sub\\)', raw_text).group(0)
test

AttributeError: 'NoneType' object has no attribute 'group'

In [19]:
test2 = re.search(r'(?<=Gained).*', raw_text).group(0)
test2

" for Gentle Whispering ASMR ',"

In [23]:
ds = "10-12-1999alksdjfalksdjflaksdjflaksdjf;alksdjf;alkjd"
date = re.search(r'^(.*?)\d\d\d\d', ds).group(0)
date

'10-12-1999'

In [24]:
ds2 = "asdlkfjasdlkfjasldkfjasldkfja#10-12-1999"
date2 = re.search(r'.+?(?=#)', ds2).group(0)
date2

'asdlkfjasdlkfjasldkfjasldkfja'

In [26]:
date3 = re.search(r'(?<=#).*', ds2).group(0)
date3

'10-12-1999'